In [1]:
import csv
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel, LabeledPoint, RandomForest

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1544511107199_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "w261_final_project_full_cluster"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

VBox()

In [4]:
full_rawRDD =  spark.read.csv("s3://w261-final-project/train.txt", sep="\t").rdd
full_dataRDD = full_rawRDD.map(lambda row: ([None if el is None else int(el) for el in row[1:14]] \
                                  + list(row[14:]), int(row[0]))).cache()

VBox()

In [5]:
# load the data we saved from the EDA. This helps us engineer the features and configure the model
frequent_feats = {}
freq_category_counts = spark.read.csv("s3://w261-final-project/e-8PURQZJJLNHJVCFY18HE4Z2P/data/freq_category_counts.csv", \
                                       header=True, sep=",").rdd.map(lambda row: row.asDict()).collect()
for row in freq_category_counts:
    total = int(row["total"])
    if total >= 10:
        key = "{}-{}".format(row["col_name"], row["category"])
        frequent_feats[key] = int(row["category_id"])

VBox()

In [6]:
num_significant_dict =  spark.read.csv("s3://w261-final-project/e-8PURQZJJLNHJVCFY18HE4Z2P/data/num_significant_categories.csv",\
                                            header=True, sep=",").rdd.map(lambda row: row.asDict()).collect()

num_significant_categories = { row["field"]: int(row["count"]) for row in num_significant_dict }

VBox()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42310)
----------------------------------------
Traceback (most recent call last):
  File "/usr/lib64/python3.4/socketserver.py", line 305, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.4/socketserver.py", line 331, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.4/socketserver.py", line 344, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.4/socketserver.py", line 673, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 263, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 238, in poll
    if func():
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 251, in authe

In [7]:
num_significant_dict

VBox()

[{'field': 'C13', 'count': '26'}, {'field': 'C2', 'count': '50'}, {'field': 'C20', 'count': '50'}, {'field': 'C1', 'count': '50'}, {'field': 'C18', 'count': '50'}, {'field': 'C23', 'count': '50'}, {'field': 'C21', 'count': '17'}, {'field': 'C14', 'count': '50'}, {'field': 'C8', 'count': '3'}, {'field': 'C6', 'count': '50'}, {'field': 'C15', 'count': '50'}, {'field': 'C9', 'count': '50'}, {'field': 'C3', 'count': '50'}, {'field': 'C24', 'count': '50'}, {'field': 'C17', 'count': '50'}, {'field': 'C22', 'count': '15'}, {'field': 'C16', 'count': '10'}, {'field': 'C11', 'count': '50'}, {'field': 'C7', 'count': '50'}, {'field': 'C4', 'count': '50'}, {'field': 'C5', 'count': '18'}, {'field': 'C0', 'count': '50'}, {'field': 'C25', 'count': '50'}, {'field': 'C10', 'count': '50'}, {'field': 'C19', 'count': '4'}, {'field': 'C12', 'count': '50'}]

In [8]:
def to_labeled(pair):
    """transform input data into the features"""
    row, label = pair
    # collect the converted values here
    vector = []
    
    for i, val in enumerate(row):
        # if this is an numerical column
        if i < 13:
            if val is None:
                val = -10
        # if this is categorical
        else:
            if val is not None:
                key = "C{}-{}".format(i - 13, val)
                # if its one of our "common" values
                if key in frequent_feats:
                    # look up its ID
                    val = frequent_feats[key]
                else:
                    # give it the special value for RARE
                    val = num_significant_categories["C" + str(i - 13)]
            else:
                # give it the special value for NULL
                val = num_significant_categories["C" + str(i - 13)] + 1
        vector.append(val)
    return LabeledPoint(label, vector)

def resample(pair):
    """sample the positive examples twice to increase their importance"""
    if pair.label == 1:
        return [pair, pair]
    else:
        return [pair]

VBox()

In [9]:

# define the hash trick to convert string features to numeric and reduce the feature space
# take a row of features and the label
# leave features 0-13 the same (already integers), convert and hash features 
def hashTrick(features, label, N):
    newFeatures = list(features[0:13])
    for i in range(14, len(features)):
        if features[i] == None:
            newFeatures += list([None])
        else:
            newFeatures += list([int(features[i], 16) % 2**N])
    yield (newFeatures, label)

VBox()

In [10]:
full_dataRDD.flatMap(lambda r: hashTrick(r[0], r[1], 16)).cache()

VBox()

PythonRDD[42] at RDD at PythonRDD.scala:52

In [11]:
fullLabeledRDD = full_dataRDD.map(to_labeled)

# set model params
categoricalFeaturesInfo = { int(feat[1:]) + 13: count + 2 for feat, count in num_significant_categories.items() }
fullTrainingData, fullValidationData = fullLabeledRDD.randomSplit([0.9, 0.1])
fullLabels = fullValidationData.map(lambda lp: lp.label).collect()

VBox()

In [12]:
final_model_rf = RandomForest.trainClassifier(fullTrainingData,
                                        categoricalFeaturesInfo={},
                                        maxBins=52,
                                        numClasses=2,
                                        maxDepth=15,
                                        numTrees=10)

VBox()

In [13]:
# Evaluate model on test instances and compute validation accuracy
final_predictions_rf = final_model_rf.predict(fullValidationData.map(lambda x: x.features))
final_preds_rf = final_predictions_rf.collect()
final_accuracy_rf = np.mean(np.array(fullLabels) == np.array(final_preds_rf))
print('accuracy = ' + str(final_accuracy_rf))

VBox()

accuracy = 0.7675660793071678

In [14]:
print(classification_report(fullLabels, final_preds_rf))
print(confusion_matrix(fullLabels, final_preds_rf))

VBox()

              precision    recall  f1-score   support

         0.0       0.78      0.96      0.86   3408223
         1.0       0.64      0.21      0.32   1175398

   micro avg       0.77      0.77      0.77   4583621
   macro avg       0.71      0.59      0.59   4583621
weighted avg       0.74      0.77      0.72   4583621

[[3267465  140758]
 [ 924631  250767]]